In [2]:
import Pkg
Pkg.add("Clustering")

   Resolving package versions...
   Installed NearestNeighbors ─ v0.4.16
   Installed Clustering ─────── v0.15.7
    Updating `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  [aaaa29a8] + Clustering v0.15.7
    Updating `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
  [aaaa29a8] + Clustering v0.15.7
  [b8a86587] + NearestNeighbors v0.4.16
Precompiling project...
  ✓ NearestNeighbors
  ✓ Clustering
  2 dependencies successfully precompiled in 24 seconds. 395 already precompiled.


In [343]:
using Clustering
using CSV
using DataFrames
using Statistics
using StatsBase
using TimeSeries
using HiddenMarkovModels


In [273]:
acwi = Matrix(CSV.read("../sb_gbi/equity_scenarios.csv",DataFrame))

99999×50 Matrix{Float64}:
  0.183608   0.040727   0.130638  …   0.310636   0.259417   0.106576
  0.057796  -0.015669   0.01473       0.181475   0.038817   0.09112
 -0.042034   0.071062   0.211914      0.271774   0.03051    0.014144
  0.048233   0.002585   0.069304      0.161997   0.150916  -0.458586
 -0.097277   0.278737   0.249458      0.054621   0.20249    0.148626
  0.168582   0.058298   0.174671  …  -0.057467  -0.037438   0.030723
  0.096997   0.214256  -0.246285      0.148997   0.192833   0.181174
  0.254872   0.054849   0.014113      0.13077    0.066608  -0.076274
  0.263792   0.213822   0.260164      0.309639   0.022386   0.268776
  0.065249  -0.206639   0.347267      0.165357   0.180862   0.298468
  ⋮                               ⋱                        
  0.304079  -0.004624   0.041851  …  -0.008278   0.136057   0.139732
  0.261117   0.094383  -0.240846      0.037133  -0.072158  -0.037526
 -0.050816   0.211829  -0.100711      0.158417  -0.021944   0.318735
  0.370563   0.279

In [274]:
inflation = Matrix(CSV.read("../sb_gbi/inflation_scenarios.csv", DataFrame))[:,2:end]

99999×50 Matrix{Float64}:
 0.035158  0.031416  0.046096  0.018379  …  0.06628   0.09403   0.093851
 0.013222  0.0123    0.043899  0.027086     0.019199  0.021789  0.020151
 0.039912  0.040263  0.035394  0.0123       0.02551   0.0123    0.0123
 0.016048  0.0123    0.0123    0.03366      0.056582  0.0123    0.0123
 0.065174  0.052422  0.071214  0.05068      0.019615  0.058837  0.048566
 0.051986  0.035766  0.026192  0.033541  …  0.052188  0.046143  0.026802
 0.040925  0.025968  0.043467  0.024486     0.0786    0.047548  0.027347
 0.0123    0.045685  0.022954  0.0123       0.019471  0.048676  0.043244
 0.0123    0.014093  0.0123    0.0123       0.037708  0.071717  0.083529
 0.0123    0.046767  0.048754  0.071964     0.058765  0.0123    0.024597
 ⋮                                       ⋱                      
 0.05639   0.034089  0.038381  0.037493  …  0.037054  0.031733  0.02699
 0.022314  0.042175  0.031607  0.039552     0.060712  0.045391  0.080075
 0.078099  0.090769  0.052886  0.02624

In [370]:
returns = stack([acwi,inflation],dims=1)

2×99999×50 Array{Float64, 3}:
[:, :, 1] =
 0.183608  0.057796  -0.042034  0.048233  …  0.119115  -0.145221  -0.025328
 0.035158  0.013222   0.039912  0.016048     0.064728   0.027103   0.0123

[:, :, 2] =
 0.040727  -0.015669  0.071062  0.002585  …  0.095772  -0.393472  0.014187
 0.031416   0.0123    0.040263  0.0123       0.058533   0.059451  0.027505

[:, :, 3] =
 0.130638  0.01473   0.211914  0.069304  …  -0.292217  0.060243  0.174296
 0.046096  0.043899  0.035394  0.0123        0.036324  0.051567  0.0123

;;; … 

[:, :, 48] =
 0.310636  0.181475  0.271774  0.161997  …  0.084165  0.142273  0.012147
 0.06628   0.019199  0.02551   0.056582     0.0123    0.04997   0.065018

[:, :, 49] =
 0.259417  0.038817  0.03051  0.150916  …  0.118251  0.023003  0.20869
 0.09403   0.021789  0.0123   0.0123       0.035531  0.0123    0.095034

[:, :, 50] =
 0.106576  0.09112   0.014144  -0.458586  …  0.292593  0.115302  0.170179
 0.093851  0.020151  0.0123     0.0123       0.045336  0.0123    0.057752

In [373]:
period_1 = returns[:,:,5] 


2×99999 Matrix{Float64}:
 -0.178384  -0.133039  0.246116  -0.156461  …  0.118621  -0.166405  0.176704
  0.043002   0.050294  0.0123     0.0123       0.031583   0.020833  0.035917

In [374]:
period_1[:,1]

2-element Vector{Float64}:
 -0.178384
  0.043002

In [375]:
n_clusters = 12
result = kmeans(period_1,n_clusters)

KmeansResult{Matrix{Float64}, Float64, Int64}([-0.05732827407859913 0.44700307672369144 … 0.12533755957667936 -0.12933353165243305; 0.04005302178782272 0.04048623328149305 … 0.023857938108288373 0.04004329876643657], [10, 12, 6, 12, 4, 7, 8, 4, 12, 11  …  9, 3, 11, 4, 6, 6, 10, 11, 12, 3], [0.0012166480857011136, 0.00011880737145460174, 0.0007990558746499704, 0.0015055901653920067, 6.668979697685082e-5, 0.0004126892444927602, 0.0004117760927620866, 0.0010743612589757083, 0.0018978129089469511, 0.0009786252272971328  …  0.001599726713853833, 0.0001742313130751877, 0.00028886581675417744, 0.00019758033866154427, 0.0022203924597280628, 0.0011765132791824173, 0.0003696126729469118, 0.0001047887537778576, 0.0017433293441404124, 0.0001269171690280041], [9825, 1929, 13123, 13876, 2200, 10225, 7387, 12368, 6140, 4964, 10580, 7382], [9822, 1929, 13127, 13880, 2200, 10225, 7387, 12364, 6141, 4964, 10583, 7377], 96.56955500357674, 100, false)

In [376]:
counts(result)

12-element Vector{Int64}:
  9825
  1929
 13123
 13876
  2200
 10225
  7387
 12368
  6140
  4964
 10580
  7382

In [377]:
centers =result.centers

2×12 Matrix{Float64}:
 -0.0573283  0.447003   0.187174   …  -0.213155  0.125338   -0.129334
  0.040053   0.0404862  0.0400764      0.040238  0.0238579   0.0400433

In [378]:
for cluster in 1:n_clusters
    println("equity: $(centers[1,cluster]), infl: $(centers[2,cluster])")
end

equity: -0.05732827407859913, infl: 0.04005302178782272
equity: 0.44700307672369144, infl: 0.04048623328149305
equity: 0.18717380193494285, infl: 0.04007637693303768
equity: 0.0639074188760809, infl: 0.037551786023053195
equity: -0.3209512604545458, infl: 0.03979883409090907
equity: 0.25065301281173524, infl: 0.04020110014669861
equity: 0.12155981981860034, infl: 0.06843289643969122
equity: 0.006372359915884843, infl: 0.0409767646392741
equity: 0.3277884077511803, infl: 0.041093245888292655
equity: -0.21315480036261059, infl: 0.04023803041901745
equity: 0.12533755957667936, infl: 0.023857938108288373
equity: -0.12933353165243305, infl: 0.04004329876643657


In [379]:
clusters = assignments(result)

99999-element Vector{Int64}:
 10
 12
  6
 12
  4
  7
  8
  4
 12
 11
  ⋮
  3
 11
  4
  6
  6
 10
 11
 12
  3

In [380]:
cluster_means = zeros((2,n_clusters))

for cluster in 1:n_clusters
    cluster_means[:,cluster] = mean(period_1[:,clusters .== cluster],dims=2) 
end


for cluster in 1:n_clusters
    println("$(cluster_means[1,cluster]), $(cluster_means[2,cluster])")
end

-0.05728395735368964, 0.04004270798982135
0.44700307672369144, 0.04048623328149305
0.18718772696791852, 0.04007951428789038
0.06394942432977824, 0.0375427498558647
-0.3209512604545458, 0.03979883409090907
0.25065990826405793, 0.04019924479217538
0.1215678191417356, 0.06843384337349384
0.006420992399741277, 0.04098200137451366
0.3277946592833874, 0.04109597198697153
-0.21315480036261059, 0.04023803041901745
0.12537072277882774, 0.023857441776939114
-0.12930915659712794, 0.0400514490652943


In [381]:
probs = counts(result)/sum(counts(result))

12-element Vector{Float64}:
 0.0982509825098251
 0.01929019290192902
 0.13123131231312313
 0.13876138761387613
 0.022000220002200022
 0.10225102251022511
 0.07387073870738707
 0.12368123681236812
 0.06140061400614006
 0.04964049640496405
 0.10580105801058011
 0.07382073820738208

In [382]:
scenario_period1 = DataFrame(Equity=cluster_means[1,:].*100,EDO=cluster_means[2,:].*100,Probs=probs*100)

Row,Equity,EDO,Probs
,Float64,Float64,Float64
1,-5.7284,4.00427,9.8251
2,44.7003,4.04862,1.92902
3,18.7188,4.00795,13.1231
4,6.39494,3.75427,13.8761
5,-32.0951,3.97988,2.20002
6,25.066,4.01992,10.2251
7,12.1568,6.84338,7.38707
8,0.642099,4.0982,12.3681
9,32.7795,4.1096,6.14006


In [384]:
function cluster(data, n_clusters)
    dt = fit(ZScoreTransform, data, dims=2)
    standarized = StatsBase.transform(dt,data)

    clusters = kmeans(standarized,n_clusters)

    destandarised = StatsBase.reconstruct(dt, clusters.centers)

    probs = counts(clusters)/sum(counts(clusters))

    return destandarised, probs
end

grid, probs = cluster(period_1, 6)
display(DataFrame(grid, :auto))
display(probs)

Row,x1,x2,x3,x4,x5,x6
,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0881188,0.274149,0.0731374,-0.0855874,0.203957,-0.154778
2,0.0467501,0.0265028,0.0176953,0.0707203,0.074317,0.0272823


6-element Vector{Float64}:
 0.21822218222182221
 0.16828168281682818
 0.22751227512275124
 0.10967109671096711
 0.13432134321343214
 0.14199141991419914

In [386]:
function cluster_moments(data, n_clusters)
    dt = fit(ZScoreTransform, data, dims=2)
    standarized = StatsBase.transform(dt,data)

    clusters = kmeans(standarized,n_clusters)
    c = assignments(clusters)

    cluster_means = zeros(2,n_clusters)
    cluster_cov = zeros(2,2,n_clusters)

    for cluster in 1:n_clusters
        cluster_means[:,cluster] = mean(period_1[:,c .== cluster],dims=2) 
        cluster_cov[:,:,cluster] = cov(period_1[:,c .== cluster],dims=2) 
    end

    return  cluster_means, cluster_cov 
end

means, c_cov = cluster_moments(period_1,6)


([0.07404183555555549 -0.08783945823886836 … 0.08869206327026127 -0.15369317981250386; 0.017707834279426234 0.07030056125872819 … 0.0467784773903881 0.02699530337632972], [0.004047449913862722 -2.009345627082811e-5; -2.009345627082811e-5 4.3294890262537256e-5;;; 0.00880723928440754 0.00014654376069173224; 0.00014654376069173224 0.0002127899409253919;;; 0.008491863872584972 -0.00031481945479353295; -0.00031481945479353295 0.00021041981014466697;;; 0.005755761465466855 0.00012266710813201942; 0.00012266710813201942 0.00015373884907403704;;; 0.004511829560240062 1.3000538365881234e-5; 1.3000538365881234e-5 6.818617975421938e-5;;; 0.007349100043782342 -5.9232574602924525e-5; -5.9232574602924525e-5 0.0001500720730855962])

In [387]:
function guess_init(n_scenarios)
    regimes_probs = rand(1:100,n_scenarios)
    return regimes_probs/sum(regimes_probs)
end

function guess_tmatrix(n_scenarios)
    regimes_probs = rand(1:100,n_scenarios,n_scenarios)
    return regimes_probs ./ sum(regimes_probs,dims=2)
end
function guess_distribution()
    corr = Matrix(1.0I,2,2)
    sigma = [rand(Uniform(0,0.25)),rand(Uniform(0,0.035))]
    means = [rand(Uniform(-0.5,0.5)),rand(Uniform(-0.01,0.12))]
    cov = cor2cov(corr,sigma)
    return MvNormal(means,cov)
end

using LinearAlgebra, Distributions

In [388]:
n_clusters = 6
init_guess = guess_init(n_clusters)
guess_matrix = guess_tmatrix(n_clusters)
guess_dist = [guess_distribution() for s in 1:n_clusters]
hmm_guess = HMM(init_guess, guess_matrix, guess_dist);
println(hmm_guess)
hmm_est_concat, _ = baum_welch(hmm_guess, eachcol(period_1));
println(hmm_est_concat)

Hidden Markov Model with:
 - initialization: [0.20161290322580644, 0.10483870967741936, 0.11559139784946236, 0.2553763440860215, 0.25268817204301075, 0.06989247311827956]
 - transition matrix: [0.10747663551401869 0.1425233644859813 0.23130841121495327 0.15654205607476634 0.17990654205607476 0.1822429906542056; 0.12340425531914893 0.04680851063829787 0.11914893617021277 0.17872340425531916 0.2851063829787234 0.24680851063829787; 0.11971830985915492 0.2112676056338028 0.31338028169014087 0.08450704225352113 0.04225352112676056 0.22887323943661972; 0.2553191489361702 0.24113475177304963 0.05673758865248227 0.20567375886524822 0.09219858156028368 0.14893617021276595; 0.07216494845360824 0.2852233676975945 0.24054982817869416 0.048109965635738834 0.024054982817869417 0.32989690721649484; 0.11811023622047244 0.2283464566929134 0.12598425196850394 0.29133858267716534 0.16535433070866143 0.07086614173228346]
 - observation distributions: [FullNormal(
dim: 2
μ: [0.05676166463983867, 0.07508278

In [ ]:
Hidden Markov Model with:
 - initialization: [1.979049851941424e-19, 0.042838672873160705, 0.10125177974650953, 0.0, 0.8559095473803299, 3.653897232409191e-158]
 - transition matrix: [0.07620753965609238 0.23289935052604743 0.1716192003548877 0.19339678218221307 0.28035575650033245 0.04552137078042694; 0.0894578572482835 0.09786921630111085 0.08957733527771651 0.19786220746397828 0.476661123853365 0.04857225985554584; 0.07527387087056006 0.23589104070157932 0.17817435696543032 0.20264058545437685 0.2630083417575164 0.04501180425053706; 0.08323880286425736 0.21259149805910554 0.1301582508039468 0.18856126354370026 0.3339741989560596 0.051475985772930354; 0.05849003451946668 0.35387428637894447 0.1902226475049301 0.16489689558522258 0.17806232976486175 0.05445380624657436; 0.06292253681862546 0.20589413642400328 0.1358136932576485 0.17997319405040463 0.3719086421509142 0.043487797298404116]
 - observation distributions: [FullNormal(
dim: 2
μ: [0.10808698710096054, 0.06878567242642063]
Σ: [0.010435002332962077 2.7994758891344456e-5; 2.7994758891344456e-5 0.00016500659831926377]
)
, FullNormal(
dim: 2
μ: [0.07348100665422401, 0.06289975918976139]
Σ: [0.0323402514404914 2.5632342060658892e-5; 2.5632342060658892e-5 0.00047564270279591784]
)
, FullNormal(
dim: 2
μ: [0.0245430252282453, 0.02788437062689451]
Σ: [0.024838119548492157 -0.00022046965113398274; -0.00022046965113398274 5.2250702702532145e-5]
)
, FullNormal(
dim: 2
μ: [0.07806797401757119, 0.0123000000000026]
Σ: [0.025458899985847366 -5.691604468150379e-31; -5.691604468150379e-31 6.7618226733876e-30]
)
, FullNormal(
dim: 2
μ: [0.10194952239669458, 0.04309385804206467]
Σ: [0.021483970377062823 -0.0003852380920975836; -0.0003852380920975836 0.00016478059203254474]
)
, FullNormal(
dim: 2
μ: [0.06797518776982549, 0.016037644731348326]
Σ: [0.024718082163571024 2.080462778412218e-6; 2.080462778412218e-6 4.79227716133304e-6]
)
]




In [389]:
n_clusters = 6
guess_dist = [MvNormal(means[:,s],c_cov[:,:,s]) for s in 1:n_clusters]
init_guess = guess_init(n_clusters)
guess_matrix = guess_tmatrix(n_clusters)
hmm_guess = HMM(init_guess, guess_matrix, guess_dist);
println(hmm_guess)
hmm_est_concat, _ = baum_welch(hmm_guess, eachcol(period_1));
println(hmm_est_concat)

Hidden Markov Model with:
 - initialization: [0.07228915662650602, 0.04216867469879518, 0.2680722891566265, 0.24096385542168675, 0.1686746987951807, 0.20783132530120482]
 - transition matrix: [0.11278195488721804 0.041353383458646614 0.2593984962406015 0.21052631578947367 0.2932330827067669 0.08270676691729323; 0.3003412969283277 0.15699658703071673 0.29692832764505117 0.08873720136518772 0.10238907849829351 0.05460750853242321; 0.006172839506172839 0.25925925925925924 0.24691358024691357 0.2006172839506173 0.2777777777777778 0.009259259259259259; 0.3074074074074074 0.21851851851851853 0.044444444444444446 0.07777777777777778 0.07407407407407407 0.2777777777777778; 0.1836734693877551 0.11564625850340136 0.2018140589569161 0.20861678004535147 0.2108843537414966 0.07936507936507936; 0.20860927152317882 0.026490066225165563 0.2251655629139073 0.1357615894039735 0.32450331125827814 0.07947019867549669]
 - observation distributions: [FullNormal(
dim: 2
μ: [0.07404183555555549, 0.01770783427

In [ ]:
Hidden Markov Model with:
 - initialization: [0.07228915662650602, 0.04216867469879518, 0.2680722891566265, 0.24096385542168675, 0.1686746987951807, 0.20783132530120482]
 - transition matrix: [0.11278195488721804 0.041353383458646614 0.2593984962406015 0.21052631578947367 0.2932330827067669 0.08270676691729323; 0.3003412969283277 0.15699658703071673 0.29692832764505117 0.08873720136518772 0.10238907849829351 0.05460750853242321; 0.006172839506172839 0.25925925925925924 0.24691358024691357 0.2006172839506173 0.2777777777777778 0.009259259259259259; 0.3074074074074074 0.21851851851851853 0.044444444444444446 0.07777777777777778 0.07407407407407407 0.2777777777777778; 0.1836734693877551 0.11564625850340136 0.2018140589569161 0.20861678004535147 0.2108843537414966 0.07936507936507936; 0.20860927152317882 0.026490066225165563 0.2251655629139073 0.1357615894039735 0.32450331125827814 0.07947019867549669]
 - observation distributions: [FullNormal(
dim: 2
μ: [0.07404183555555549, 0.017707834279426234]
Σ: [0.004047449913862722 -2.009345627082811e-5; -2.009345627082811e-5 4.3294890262537256e-5]
)
, FullNormal(
dim: 2
μ: [-0.08783945823886836, 0.07030056125872819]
Σ: [0.00880723928440754 0.00014654376069173224; 0.00014654376069173224 0.0002127899409253919]
)
, FullNormal(
dim: 2
μ: [0.20257526673118503, 0.07453515454477798]
Σ: [0.008491863872584972 -0.00031481945479353295; -0.00031481945479353295 0.00021041981014466697]
)
, FullNormal(
dim: 2
μ: [0.27460819369047706, 0.0265884376785701]
Σ: [0.005755761465466855 0.00012266710813201942; 0.00012266710813201942 0.00015373884907403704]
)
, FullNormal(
dim: 2
μ: [0.08869206327026127, 0.0467784773903881]
Σ: [0.004511829560240062 1.3000538365881234e-5; 1.3000538365881234e-5 6.818617975421938e-5]
)
, FullNormal(
dim: 2
μ: [-0.15369317981250386, 0.02699530337632972]
Σ: [0.007349100043782342 -5.9232574602924525e-5; -5.9232574602924525e-5 0.0001500720730855962]
)
]
Hidden Markov Model with:
 - initialization: [0.0, 0.0003652606152262479, 2.2945688795404057e-57, 3.3805080174130594e-70, 8.660246134457614e-29, 0.9996347393847738]
 - transition matrix: [0.18856126354370084 0.14636780585750833 0.1562464141966821 0.1442212275648889 0.2739207422012418 0.09068254663597791; 0.1791515401262556 0.1420714082569363 0.17221590479778442 0.13840637250559018 0.2779783493915593 0.09017642492187423; 0.1875183822176813 0.1407208269142285 0.16057202743891974 0.1455896470321112 0.2744028267872247 0.09119628960983453; 0.179843965745754 0.1436529059891184 0.16334457994534696 0.15057603164904593 0.2691753789442568 0.09340713772647788; 0.18866183116917526 0.14298430220263303 0.16461097490836224 0.1473425435885883 0.2605232584422193 0.09587708968902181; 0.18528188978926663 0.1370165882955184 0.16664845563416175 0.13569611034594167 0.2908288404956383 0.0845281154394732]
 - observation distributions: [FullNormal(
dim: 2
μ: [0.07806797401757337, 0.012299999999998612]
Σ: [0.025458899985847896 9.566816257500546e-31; 9.566816257500546e-31 1.9259299443872327e-30]
)
, FullNormal(
dim: 2
μ: [-0.06872948963493297, 0.060072334908710716]
Σ: [0.01886036202104725 0.00032022050049567964; 0.00032022050049567964 0.00029748675193451597]
)
, FullNormal(
dim: 2
μ: [0.1871040427524066, 0.06712888733193507]
Σ: [0.01898617947964987 -0.0010251852582424028; -0.0010251852582424028 0.0004755807513737602]
)
, FullNormal(
dim: 2
μ: [0.16391899438978022, 0.024889201116229002]
Σ: [0.012372539065645555 0.000122834797092125; 0.000122834797092125 6.028174072721973e-5]
)
, FullNormal(
dim: 2
μ: [0.09952784688227974, 0.045241664337006715]
Σ: [0.010593912255488813 0.00018625992747647952; 0.00018625992747647952 0.00018973687245039524]
)
, FullNormal(
dim: 2
μ: [-0.08678884625348669, 0.027994248807185866]
Σ: [0.015096807008931557 -6.732927449909418e-5; -6.732927449909418e-5 8.523686665116547e-5]
)
]


In [317]:
source_data = TimeArray(CSV.File("assets_data_03.24.csv", delim=';', dateformat=Dict("Date" => "dd.mm.yyyy"), decimal=','),timestamp=:Date)
returns = percentchange(source_data, :log)


615×5 TimeArray{Union{Missing, Float64}, 2, Date, Matrix{Union{Missing, Float64}}} 1973-01-31 to 2024-03-31
┌────────────┬────────────┬─────────────┬──────────────┬──────────────┬─────────
│            │ ACWI_Net   │ USDPLN      │ ACWI_Net_PLN │ CPI_USA      │ CPI_PL ⋯
├────────────┼────────────┼─────────────┼──────────────┼──────────────┼─────────
│ 1973-01-31 │    missing │     missing │      missing │   0.00235067 │     mi ⋯
│ 1973-02-28 │    missing │     missing │      missing │   0.00701905 │     mi ⋯
│ 1973-03-31 │    missing │     missing │      missing │   0.00927727 │     mi ⋯
│ 1973-04-30 │    missing │     missing │      missing │   0.00690598 │     mi ⋯
│ 1973-05-31 │    missing │     missing │      missing │   0.00685862 │     mi ⋯
│ 1973-06-30 │    missing │     missing │      missing │    0.0068119 │     mi ⋯
│ 1973-07-31 │    missing │     missing │      missing │   0.00225501 │     mi ⋯
│ 1973-08-31 │    missing │     missing │      missing │    0.0179014 │     mi ⋯
│

In [318]:
returns_yearly = percentchange(collapse(source_data,year, last))

52×5 TimeArray{Union{Missing, Float64}, 2, Date, Matrix{Union{Missing, Float64}}} 1973-12-31 to 2024-03-31
┌────────────┬────────────┬────────────┬──────────────┬────────────┬────────────
│            │ ACWI_Net   │ USDPLN     │ ACWI_Net_PLN │ CPI_USA    │ CPI_PLN   ⋯
├────────────┼────────────┼────────────┼──────────────┼────────────┼────────────
│ 1973-12-31 │    missing │    missing │      missing │  0.0870605 │     missi ⋯
│ 1974-12-31 │    missing │    missing │      missing │   0.123377 │     missi ⋯
│ 1975-12-31 │    missing │    missing │      missing │  0.0693605 │     missi ⋯
│ 1976-12-31 │    missing │    missing │      missing │  0.0486505 │     missi ⋯
│ 1977-12-31 │    missing │    missing │      missing │  0.0670123 │     missi ⋯
│ 1978-12-31 │    missing │    missing │      missing │   0.090177 │     missi ⋯
│ 1979-12-31 │    missing │    missing │      missing │   0.132939 │     missi ⋯
│ 1980-12-31 │    missing │    missing │      missing │   0.125161 │     missi ⋯
│ 

In [319]:
val = eachcol(values(from(returns_yearly,Date(2003,02,01))))

[percentile(a,[5, 25, 50, 75, 95]) for a in val]

5-element Vector{Vector{Float64}}:
 [-0.1791646535637412, 0.050866825980199704, 0.13951720841975523, 0.21889844750843207, 0.3361762844747671]
 [-0.16789023904324368, -0.08309423208140795, 0.01405036560292702, 0.08138626868521559, 0.17291389833351883]
 [-0.11150993397204947, 0.03390535467137583, 0.09733228505343647, 0.19875932356089762, 0.30291930341482903]
 [0.007308352896474506, 0.015616012312351955, 0.02091860212971578, 0.03182293580839392, 0.06335757465785029]
 [-0.005813596241253867, 0.00972740828317448, 0.02302606076369674, 0.03842710017413295, 0.08567916010546725]

In [320]:
acwi_cpipln = identity.(values(from(returns[:ACWI_Net_PLN,:CPI_PLN],Date(2003,02,01))))

254×2 Matrix{Float64}:
  0.0051036     0.000927653
  0.0430686     0.00277912
  0.0118412     0.0018489
  0.0300994     0.0
  0.0670022    -0.000924682
  0.0132941    -0.00370334
  0.0495565    -0.00371711
  0.000195397   0.0046439
  0.0782452     0.00554541
 -0.0161735     0.00276122
  ⋮            
  0.0208096    -0.00221661
  0.000986609   0.0
  0.0154043    -0.00389282
 -0.066645      0.00278164
  0.0356998     0.00719694
  0.0310713     0.00110186
  0.0233511     0.00384933
  0.0394925     0.00274048
  0.0298368     0.00218699

In [321]:
transpose(acwi_cpipln)

2×254 transpose(::Matrix{Float64}) with eltype Float64:
 0.0051036    0.0430686   0.0118412  …  0.0233511   0.0394925   0.0298368
 0.000927653  0.00277912  0.0018489     0.00384933  0.00274048  0.00218699

In [322]:
state, probs = cluster(transpose(acwi_cpipln),4)
display(state*12)
display(probs)


2×4 Matrix{Float64}:
 0.506915   -0.741961   -0.0241622  -0.0317024
 0.0235401   0.0357811   0.0073365   0.143168

4-element Vector{Float64}:
 0.33858267716535434
 0.09055118110236221
 0.44881889763779526
 0.1220472440944882

In [323]:
acwi_cpiusd = identity.(values(from(returns[:ACWI_Net,:CPI_USA],Date(1988,01,01))))

state, probs = cluster(transpose(acwi_cpiusd),3)

display(state)
display(probs)

2×3 Matrix{Float64}:
 0.0208741    0.0300136    -0.0488946
 0.00509747  -0.000185102   0.00239701

3-element Vector{Float64}:
 0.34022988505747126
 0.39770114942528734
 0.2620689655172414